In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

!pip install pytorch_transformers

In [0]:
import pandas as pd
import numpy as np
import unicodedata
import torch
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from bs4 import BeautifulSoup
from pytorch_transformers import BertTokenizer, BertModel

import re

df = pd.read_csv('/content/gdrive/My Drive/IMDB_Dataset.csv', encoding='utf-8')


In [0]:
# Sentiment 1, 0으로 바꾸기
df['sentiment'] = (df['sentiment'] == 'positive').astype(int)

In [0]:
# 문장 단위로 쪼개기
df.review = df.review.map(sent_tokenize)
df_reviews = df_reviews.stack().reset_index(level=1, drop=True).to_frame('review_single')
df_pre = df[["sentiment"]].merge(df_reviews, left_index=True, right_index=True, how='left')
raw_reviews = df_pre.values

In [0]:
# 클렌징
stop = stopwords.words('english')
lmtzr = WordNetLemmatizer()


def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def clean_review(raw_review: str) -> str:
    # 1. unicode to ASCII
    raw_review = unicodeToAscii(raw_review)
    
    # 2. Remove HTML
    review_text = BeautifulSoup(raw_review, "lxml").get_text()
    
    # 3. Tokenize:
    review_text = [word for word in nltk.word_tokenize(review_text)]
    
    # 4. Remove Stop Words
    review_text = [word for word in review_text if word not in stop and len(word) >= 3]
    
    # 5. lemmatization
    review_text = [lmtzr.lemmatize(word) for word in review_text]

    return " ".join(review_text)
  
reviews = []
  
for r in raw_reviews:
  new_r = clean_review(r[1])
  if(len(new_r) > 5 and new_r[0].isalpha()):
    reviews.append((new_r, r[0]))

In [0]:
import pickle
import gzip

with gzip.open('/content/gdrive/My Drive/pre_reviews.pickle', 'wb') as f:
    pickle.dump(reviews, f)